# 第 14 章　可迭代的对象、迭代器和生成器

迭代是数据处理的基石。扫描内存中放不下的数据集时，我们要找到一
种惰性获取数据项的方式，即按需一次获取一个数据项。这就是迭代器
模式（Iterator pattern）。本章说明 Python 语言是如何内置迭代器模式
的，这样就避免了自己手动去实现。

与 Lisp不同，Python 没有宏，因此为了抽象出迭代器模式，需要改动语言本身。
为此，Python 2.2（2001 年）加入了 yield 关键字。
这个关键字用于构建生成器（generator），其作用与迭代器一样。

所有生成器都是迭代器，因为生成器完全实现了迭代器接口。
但，**迭代器**用于从集合中**抽取**元素；而**生成器**用于”凭空“**生成**元素。
python社区不区分两者区别。

在 Python 中，所有集合都可以迭代。在 Python 语言内部，迭代器用于支持：
- for 循环
- 构建和扩展集合类型
- 逐行遍历文本文件
- 列表推导、字典推导和集合推导
- 元组拆包
- 调用函数时，使用 * 拆包实参

本章内容：
- 语言内部使用 iter(...) 内置函数处理可迭代对象的方式
- 如何使用 Python 实现经典的迭代器模式
- 详细说明生成器函数的工作原理
- 如何使用生成器函数或生成器表达式代替经典的迭代器
- 如何使用标准库中通用的生成器函数
- 如何使用 yield from 语句合并生成器
- 案例分析：在一个数据库转换工具中使用生成器函数处理大型数据集
- 为什么生成器和协程看似相同，实则差别很大，不能混淆

## 14.1 单词序列案例

In [ ]:
from sentence import Sentence

s = Sentence('"The time has come," the Walrus said,')
s

In [3]:
for w in s:
    print(w)

The
time
has
come
the
Walrus
said


In [4]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

In [5]:
s[3]

'come'

### 序列可以迭代的原因：iter函数

解释器需要迭代对象 x 时，会自动调用 iter(x)。
内置的 iter 函数有以下作用。
- (1) 检查对象是否实现了 __iter__ 方法，如果实现了就调用它，获取
一个迭代器。
- (2) 如果没有实现 __iter__ 方法，但是实现了 __getitem__ 方法，
Python 会创建一个迭代器，尝试按顺序（从索引 0 开始）获取元素。
- (3) 如果尝试失败，Python 抛出 TypeError 异常，通常会提示“C object
is not iterable”（C 对象不可迭代），其中 C 是目标对象所属的类。

In [1]:
# 判断一个类是否是可以迭代的，建议使用下面的方法，而不要使用issubclass(abc.Iterable)
# 因为iter()只要实现__getitem__或__iter__其中一个就可以，而使用subclass检查必须实现__iter__
iter(s)

NameError: name 's' is not defined

- 如果对象实现了能返回迭代器的 __iter__ 方法，那么对象就是可迭代的。序列都可以迭代；
- 实现了 __getitem__ 方法，而且其参数是从零开始的索引，这种对象也可以迭代。

我们要明确可迭代的对象和迭代器之间的关系：Python 从可迭代的对象中获取迭代器。

In [10]:
s = 'ABC'
i = iter(s)
while True:
    try:
        print(next(i))
    except StopIteration:
        del i
        break

A
B
C


## 　Sentence类第2版：典型的迭代器

因此，迭代器可以迭代，但是可迭代的对象不是迭代器。

除了 \_\_iter__ 方法之外，你可能还想在 Sentence 类中实现
\_\_next__ 方法，让 Sentence 实例既是可迭代的对象，也是自身的
迭代器。可是，这种想法非常糟糕。根据有大量 Python 代码审查经验的
Alex Martelli 所说，这也是常见的反模式。

迭代器模式可用来：
- 访问一个聚合对象的内容而无需暴露它的内部表示
- 支持对聚合对象的多种遍历
- 为遍历不同的聚合结构提供一个统一的接口（即支持多态迭代）

可迭代的对象一定不能是自身的迭代器。也就是说，可迭代的对象
必须实现 \_\_iter__ 方法，但不能实现 \_\_next__ 方法。

In [ ]:
import re
import reprlib

"""
第二版
实现迭代器
"""

RE_WORD = re.compile('\w+')


class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return "Sentence(%s)" % reprlib.repr(self.text)

    def __iter__(self):
        return SentenceIterator(self.words)


class SentenceIterator:
"""
    Sentence类的迭代器
"""
    def __init__(self, words):
        self.words = words
        self.index = 0

    # 返回下一个可用的元素，如果没有元素了，抛出 StopIteration异常。
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word

    # 返回 self，以便在应该使用可迭代对象的地方使用迭代器，例如在 for 循环中。
    def __iter__(self):
        return self



"""
因此，迭代器可以迭代，但是可迭代的对象不是迭代器。

除了 __iter__ 方法之外，你可能还想在 Sentence 类中实现
__next__ 方法，让 Sentence 实例既是可迭代的对象，也是自身的
迭代器。可是，这种想法非常糟糕。根据有大量 Python 代码审查经验的
Alex Martelli 所说，这也是常见的反模式。



迭代器模式可用来：
- 访问一个聚合对象的内容而无需暴露它的内部表示
- 支持对聚合对象的多种遍历
- 为遍历不同的聚合结构提供一个统一的接口（即支持多态迭代）
"""

## 　Sentence类第3版：生成器函数

In [ ]:
import re
import reprlib

"""
第三版
符合Python习惯的实现迭代器
"""

RE_WORD = re.compile('\w+')

class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return "Sentence(%s)" % reprlib.repr(self.text)

    def __iter__(self):
        for word in self.words:
            yield word
        return  # 这个 return 语句不是必要的；这个函数可以直接“落空”，自动返
                # 回。不管有没有 return 语句，生成器函数都不会抛出
                # StopIteration 异常，而是在生成完全部值之后会直接退出。

    """
    # 也可使用更加简洁的方法：
    def __iter__(self):
        return iter(self.words) # 调用 __iter__ 方法得到的就是迭代器
    """

只要 Python 函数的定义体中有 yield 关键字，该函数就是生成器函
数。调用生成器函数时，会返回一个生成器对象。也就是说，生成器函
数是生成器工厂。

In [11]:
# 理解yield关键字

def gen_123(): #只要 Python 函数中包含关键字 yield，该函数就是生成器函数
    yield 1
    yield 2
    yield 3

gen_123
# <function __main__.gen_123()>

<function __main__.gen_123()>

In [12]:
gen_123()
# <generator object gen_123 at 0x01467108>

<generator object gen_123 at 0x01467108>

In [13]:
for i in gen_123():
    print(i)


1
2
3


In [14]:
g = gen_123()
next(g)

1

In [15]:
next(g)


2

In [16]:
next(g)

3

In [20]:
next(g)
# StopIteration

StopIteration: 

生成器函数会创建一个生成器对象，包装生成器函数的定义体。把生成
器传给 next(...) 函数时，生成器函数会向前，执行函数定义体中的
下一个 yield 语句，返回产出的值，并在函数定义体的当前位置暂
停。最终，函数的定义体返回时，外层的生成器对象会抛出
StopIteration 异常——这一点与迭代器协议一致。

In [21]:
# 生成器函数定义体的执行过程：
def gen_AB():
    print('start')
    yield "A"
    print('continue')
    yield 'B'
    print('end')

for c in gen_AB():
    print('-->', c)

start
--> A
continue
--> B
end


In [26]:
next(gen_AB())

start


'A'

## Sentence类第4版：惰性实现

懒惰的反义词是急迫，其实，惰性求值（lazy evaluation）
和及早求值（eager evaluation）是编程语言理论方面的技术术语。
目前实现的几版 Sentence 类都不具有惰性，因为 \_\_init__ 方法急
迫地构建好了文本中的单词列表，然后将其绑定到 self.words 属性
上。这样就得处理整个文本，列表使用的内存量可能与文本本身一样多
（或许更多，这取决于文本中有多少非单词字符）。如果只需迭代前几
个单词，大多数工作都是白费力气。

In [ ]:
import re
import reprlib

"""
第四版
惰性实现
"""

RE_WORD = re.compile('\w+')

class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return "Sentence(%s)" % reprlib.repr(self.text)

    """
    def __iter__(self):
        for word in self.words:
            yield word
        return
    """
    def __iter__(self):
        for match in RE_WORD.finditer(self.text): # finditer 函数构建一个迭代器，包含 self.text 中匹配
                                                # RE_WORD 的单词，产出 MatchObject 实例。
            yield match.group()  # match.group() 方法从 MatchObject 实例中提取匹配正则表达
                                 # 式的具体文本。

## Sentence类第5版：生成器表达式
生成器表达式可以理解为列表推导的惰性版本：不会迫切地构建列表，
而是返回一个生成器，按需惰性生成元素。也就是说，如果列表推导是
制造列表的工厂，那么生成器表达式就是制造生成器的工厂。

In [38]:
# 在列表推导中使用 gen_AB 生成器函数
def gen_AB():
    print('start')
    yield "A"
    print('continue')
    yield 'B'
    print('end')

res1 = [x*3 for x in gen_AB()] # 列表推导
res1

start
continue
end


['AAA', 'BBB']

In [39]:
for x in res1:
    print("-->>", x)

-->> AAA
-->> BBB


In [42]:
res2 = (x*3 for x in gen_AB()) # 生成器表达式
res2

<generator object <genexpr> at 0x0148D108>

In [43]:
for x in res2:
    print("-->>", x)

start
-->> AAA
continue
-->> BBB
end


res2 是一个生成器对象。

只有 for 循环迭代 res2 时，gen_AB 函数的定义体才会真正执
行。for 循环每次迭代时会隐式调用 next(res2)，前进到 gen_AB
函数中的下一个 yield 语句。注意，gen_AB 函数的输出与 for 循环
中 print 函数的输出夹杂在一起。

In [ ]:
import re
import reprlib

"""
第四版
惰性实现
"""
RE_WORD = re.compile('\w+')

class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return "Sentence(%s)" % reprlib.repr(self.text)
    """ 第一版
    def __getitem__(self, item):
        return self.words[item]

    def __len__(self):
        return len(self.words)
    """
    """第二版
    def __iter__(self):
        return SentenceIterator(self.words)
    """

    """第三版
    def __iter__(self):
        for word in self.words:
            yield word
        return
    """
    """第四版
    def __iter__(self):
        for match in RE_WORD.finditer(self.text):
            yield match.group()
    """
    # 第五版 使用生成器表达式惰性生成迭代器！
    def __iter__(self):
        return (yield match.group() for match in RE_WORD.finditer(self.text))


## 标准库中的生成器函数

### 1 过滤生成器函数

In [ ]:
def vowel(c):
    return c.lower() in 'aeiou'


In [2]:
# filter(predicate, it) 把it中的各个元素传递给predicate，如果predicate(iten)返回真值，
# 那么产出对应的元素；如果返回是None，那么只产出真值元素。
list(filter(vowel, 'Aardvark'))

['A', 'a', 'a']

In [4]:
import itertools
# 与filter一样，只是返回None时的值
list(itertools.filterfalse(vowel, 'Aardvark'))

['r', 'd', 'v', 'r', 'k']

In [5]:
# 处理it，跳过predicate的计算结果为真的元素，然后产出剩下的各个元素（不再进一步检查）
list(itertools.dropwhile(vowel, 'Aardvark'))

['r', 'd', 'v', 'a', 'r', 'k']

In [6]:
# 处理it，返回predicate的计算结果为真的元素，然后立即停止，不再继续检查
list(itertools.takewhile(vowel, 'Aardvark'))

['A', 'a']

In [7]:
# 并行处理两个序列，第二个序列返回True时，返回第一个序列的对应元素
list(itertools.compress('Aardvark',(1,0,1,0,0)))

['A', 'r']

In [8]:
# 切片惰性返回
list(itertools.islice('Aardvark', 4))

['A', 'a', 'r', 'd']

In [13]:
list(itertools.islice('Aardvark', 4, 21))

['v', 'a', 'r', 'k']

In [12]:
list(itertools.islice('Aardvark', 4, 21, 2))

['v', 'r']

### 2 映射生成器函数

In [14]:
sample = [5,4,2,8,7,6,3,0,9,1]

In [15]:
import itertools
# 计算累计总和，如果提供了func参数，就把两个元素传入，然后将结果和下一个元素传递给它。
r = list(itertools.accumulate(sample))
r

[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]

In [16]:
r = list(itertools.accumulate(sample, min))
r


[5, 4, 2, 2, 2, 2, 2, 0, 0, 0]

In [18]:
r = list(itertools.accumulate(sample, lambda a,b : a*b))
r
# 如果不想使用匿名函数，可以用下面的方法（推荐）
import operator
r = list(itertools.accumulate(sample, operator.mul))
r

[5, 20, 40, 320, 2240, 13440, 40320, 0, 0, 0]

In [20]:
# 把第二个开始的参数传入第一个参数的函数中，并且函数要有对应个数的参数。并行处理多个序列的参数。
r = list(map(operator.mul,range(11),range(11)))
r

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [21]:
# enumerate(iterable, start=0) :返回（index，item），index的开始数组由start确定
# startmap(func, it):把it中的各个元素传给func，产出结果。输入的可迭代对象应该产生可迭代的元素iit，然后以func(*iit)
# 这种形式调用func
list(itertools.starmap(operator.mul, enumerate('albatroz', 1)))

['a', 'll', 'bbb', 'aaaa', 'ttttt', 'rrrrrr', 'ooooooo', 'zzzzzzzz']

In [24]:
# startmap(func, it):把it中的各个元素传给func，产出结果。输入的可迭代对象应该产生可迭代的元素iit，然后以func(*iit)
# 这种形式调用func
list(itertools.starmap(lambda a,b : b/a, enumerate(itertools.accumulate(sample),1)))

[5.0,
 4.5,
 3.6666666666666665,
 4.75,
 5.2,
 5.333333333333333,
 5.0,
 4.375,
 4.888888888888889,
 4.5]

### 3 合并生成器函数

In [25]:
# 先产出第一个参数的所有元素，然后在产出第二参数的所有元素，以此类推。
list(itertools.chain('ABC', range(2)))

['A', 'B', 'C', 0, 1]

In [26]:
list(itertools.chain(enumerate('ABC')))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [27]:
# 产出it生成的各个可迭代对象中的元素，一个接一个，无缝连接在一起。it应该产出可迭代的元素，例如可迭代的对象列表：
list(itertools.chain.from_iterable((enumerate('ABC'))))

[0, 'A', 1, 'B', 2, 'C']

In [29]:
# 生成合并组元的序列，但是长度为最短的
list(zip('ABC', range(3), [10,20,30,40]))

[('A', 0, 10), ('B', 1, 20), ('C', 2, 30)]

In [30]:
# 生成合并组元的序列，但是长度为最长的
list(itertools.zip_longest('ABC', range(4), [10,20,30,40,50]))

[('A', 0, 10), ('B', 1, 20), ('C', 2, 30), (None, 3, 40), (None, None, 50)]

In [31]:
# 生成合并组元的序列，但是长度为最长的，并能设置缺失元素的填充值
list(itertools.zip_longest('ABC', range(4), [10,20,30,40,50], fillvalue='Null'))

[('A', 0, 10),
 ('B', 1, 20),
 ('C', 2, 30),
 ('Null', 3, 40),
 ('Null', 'Null', 50)]

In [32]:
# 笛卡尔积
list(itertools.product('ABC', range(2)))

[('A', 0), ('A', 1), ('B', 0), ('B', 1), ('C', 0), ('C', 1)]

In [33]:
suits = 'spades hearts diamonds clubs'.split()
list(itertools.product('AJQK', suits))

[('A', 'spades'),
 ('A', 'hearts'),
 ('A', 'diamonds'),
 ('A', 'clubs'),
 ('J', 'spades'),
 ('J', 'hearts'),
 ('J', 'diamonds'),
 ('J', 'clubs'),
 ('Q', 'spades'),
 ('Q', 'hearts'),
 ('Q', 'diamonds'),
 ('Q', 'clubs'),
 ('K', 'spades'),
 ('K', 'hearts'),
 ('K', 'diamonds'),
 ('K', 'clubs')]

In [34]:
list(itertools.product('ABC'))

[('A',), ('B',), ('C',)]

In [35]:
list(itertools.product('ABC', repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

In [36]:
list(itertools.product(range(2), repeat=3))

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [37]:
rows = itertools.product("AB", range(2), repeat=2)
for row in rows:
    print(row)

('A', 0, 'A', 0)
('A', 0, 'A', 1)
('A', 0, 'B', 0)
('A', 0, 'B', 1)
('A', 1, 'A', 0)
('A', 1, 'A', 1)
('A', 1, 'B', 0)
('A', 1, 'B', 1)
('B', 0, 'A', 0)
('B', 0, 'A', 1)
('B', 0, 'B', 0)
('B', 0, 'B', 1)
('B', 1, 'A', 0)
('B', 1, 'A', 1)
('B', 1, 'B', 0)
('B', 1, 'B', 1)


### 4 其他特殊函数：能够产出更多元素的生成器

In [38]:
# 可以生成无数个元素
ct = itertools.count()
next(ct)

0

In [39]:
next(ct), next(ct), next(ct)

(1, 2, 3)

In [40]:
# 使用切片防止无限崩溃
list(itertools.islice(itertools.count(1, 0.3),3))

[1, 1.3, 1.6]

In [41]:
# 循环
cy = itertools.cycle("ABC")
next(cy)

'A'

In [42]:
list(itertools.islice(cy, 7))

['B', 'C', 'A', 'B', 'C', 'A', 'B']

In [43]:
# 没有第二参数的时候，每次都会重复输出
rp = itertools.repeat(7)
next(rp), next(rp), next(rp), next(rp)

(7, 7, 7, 7)

In [48]:
rp2 = itertools.repeat('ABC')
next(rp2)

'ABC'

In [ ]:
list(itertools.repeat(7,4))

In [45]:
list(map(operator.mul, range(10), itertools.repeat(5)))

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45]

### 5 组合学 生成器 （ combinatoric generator ）

包括：
- itertools.combinations
- itertools.combinations_with_replacement
- itertools.permutations
- itertools.product

In [49]:
# 'ABC'每两个元素的各种组合, 元素的顺序没有意义
list(itertools.combinations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [50]:
# 'ABC'每两个元素的各种组合, 包括相同元素的组合
list(itertools.combinations_with_replacement('ABC', 2))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

In [51]:
# 'ABC'每两个元素的各种组合, 元素的顺序有重要意义
list(itertools.permutations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]

In [53]:
# 笛卡尔积
list(itertools.product('ABC', repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

### 6 用于产出输入的可迭代对象中的全部元素，但重新排列

In [54]:
list(itertools.groupby('AAAABBBCCCCCC'))

[('A', <itertools._grouper at 0x7fe31b132df0>),
 ('B', <itertools._grouper at 0x7fe31b132490>),
 ('C', <itertools._grouper at 0x7fe31b132910>)]

In [55]:
for char, group in itertools.groupby('AAAABBBCCCCCC'):
    print(char, '-->', list(group))

A --> ['A', 'A', 'A', 'A']
B --> ['B', 'B', 'B']
C --> ['C', 'C', 'C', 'C', 'C', 'C']


In [65]:
animals = ['duck', 'eagle', 'rat', 'giraffe', 'bear', 'bat', 'dolphin', 'shark', 'lion', 'dog']
animals.sort(key=len)
animals

['rat',
 'bat',
 'dog',
 'duck',
 'bear',
 'lion',
 'eagle',
 'shark',
 'giraffe',
 'dolphin']

In [66]:
# 以长度进行分类
for length, group in itertools.groupby(animals, len):
    print(length, '-->', list(group))

3 --> ['rat', 'bat', 'dog']
4 --> ['duck', 'bear', 'lion']
5 --> ['eagle', 'shark']
7 --> ['giraffe', 'dolphin']


In [68]:
# 反向animals列表，竟然能够影响输出的顺序！需要进一步理解！
for length, group in itertools.groupby(reversed(animals), len):
    print(length, '-->', list(group))

7 --> ['dolphin', 'giraffe']
5 --> ['shark', 'eagle']
4 --> ['lion', 'bear', 'duck']
3 --> ['dog', 'bat', 'rat']


In [69]:
# 以首字母分类：
for name, group in itertools.groupby(animals, lambda a:a[0]):
    print(name, '-->', list(group))

r --> ['rat']
b --> ['bat']
d --> ['dog', 'duck']
b --> ['bear']
l --> ['lion']
e --> ['eagle']
s --> ['shark']
g --> ['giraffe']
d --> ['dolphin']


In [70]:
# 从输入的一个可迭代对象中产出多个生成器，每个生成器都可以产出输入的各个元素。
list(itertools.tee('ABC'))

[<itertools._tee at 0x7fe3196d9cc0>, <itertools._tee at 0x7fe31a005480>]

In [71]:
g1, g2 = itertools.tee('ABC')

In [72]:
next(g1)

'A'

In [73]:
next(g2)

'A'

In [74]:
next(g1)


'B'

In [75]:
list(g1)

['C']

In [76]:
list(g2)

['B', 'C']

In [77]:
list(zip(*itertools.tee('ABC')))

[('A', 'A'), ('B', 'B'), ('C', 'C')]

In [78]:
# 默认生成两个， 可以设置任意数量
list(itertools.tee('ABC', 5))